# Chat completion example LLaMA-2-7b-chat (CPU)
- Instantiate LlaMa-2-7b-chat in CPU
- Make inference over a custom user prompt
- Retrive answer and profile memory and time consumption

In [ ]:
!lscpu | egrep 'Model name|socket|Thread|CPU\(s\)'

Import libraries

In [ ]:
from llama import Llama

In [ ]:
# Checkpoint dir, tokenizer path and sizes.
ckpt_dir = '/app/models/llama-2-7b-chat/'
tokenizer_path = '/app/models/llama-2-7b-chat/tokenizer.model'
max_seq_len = 512
max_batch_size = 1

In [ ]:
!free -g

In [ ]:
# Creates a generator
generator = Llama.build(
    ckpt_dir=ckpt_dir,
    tokenizer_path=tokenizer_path,
    max_seq_len=max_seq_len,
    max_batch_size=max_batch_size,
    model_parallel_size=1,
    device='cpu',
)

In [ ]:
!free -g

In [ ]:
model_parameters = 7e9
bytes_per_parameter = 2
print(f"Model size in memory: {model_parameters*bytes_per_parameter/2**30:.3f} GBi")

In [ ]:
# Model architecture
generator.model

# Model parameters

In [ ]:
generator.model.params

# Chat completion

In [ ]:
# User prompt
# dialogs = [[{"role": "user", "content": "what is the recipe of mayonnaise?"}]]
# dialogs = [[{"role": "user", "content": "Resume Investor Exchange (IEX) story"}]]
dialogs = [[{"role": "user", "content": "Hi"}]]

Inputs token

In [ ]:
# Get the number of inputs tokens
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
SPECIAL_TAGS = [B_INST, E_INST, "<<SYS>>", "<</SYS>>"]

print(f"Prompt: {B_INST} {(dialogs[0][0]['content']).strip()} {E_INST}")
prompt_tokens = generator.tokenizer.encode(f"{B_INST} {(dialogs[0][0]['content']).strip()} {E_INST}",
                                           bos=True,
                                           eos=True)
print(f"Prompt tokens: {len(prompt_tokens)}")

Generating meaningful responses in the context of a chat or conversation

In [ ]:
results = generator.chat_completion(
    dialogs, 
    max_gen_len=None,
    temperature=0.6,
    top_p=0.9,
)

check the assistant output

In [ ]:
for dialog, result in zip(dialogs, results):
    for msg in dialog:
        print(f"{msg['role'].capitalize()}: {msg['content']}\n")
    print(f"> {result['generation']['role'].capitalize()}: {result['generation']['content']}")
    print("\n==================================\n")

Output tokens

In [ ]:
answer_tokens = generator.tokenizer.encode(f"{B_INST} {(result['generation']['content']).strip()} {E_INST}",
                                           bos=True,
                                           eos=True)
print(f"Answer tokens: {len(answer_tokens)}")

# generator.tokenizer.decode(answer_tokens)